# Prueba de concepto

filters: min. maps = 58

In [46]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import cloudscraper

In [47]:
# En vez de utilizar request creamos un instancia de un scraper que simula un navegador en Python
# para evadir la verificación de cloudflare

scraper = cloudscraper.create_scraper(
    browser={
        "browser": "chrome",
        "platform": "windows",
    },
)

# Top jugadores

All matches, Last 12 months, Top 20, All maps, Both sides

In [52]:
url = "https://www.hltv.org/stats/players?startDate=2023-10-21&endDate=2024-10-21&rankingFilter=Top20"
response = scraper.get(url)

print(f"The status code is {response.status_code}")

The status code is 200


In [58]:
soup = bs(response.text)
tables = soup.find_all(class_="stats-table player-ratings-table")

df = pd.DataFrame(columns=["Player", "Teams", "Maps", "Rounds", "K-D Diff", "K/D", "Rating"])

for row in tables[0].find_all('tr')[1:]:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]
    
    new_row = pd.DataFrame([cols], columns=df.columns)

    df = pd.concat([df, new_row], ignore_index=True, axis=0)

In [61]:
df.head()

,Player,Teams,Maps,Rounds,K-D Diff,K/D,Rating
0,ZywOo,,130,2873,+772,1.46,1.34
1,donk,,135,2843,+623,1.32,1.32
2,m0NESY,,139,2995,+695,1.39,1.28
3,XANTARES,,130,2916,+240,1.12,1.17
4,EliGE,,123,2728,+180,1.09,1.14


# Top equipos

All matches, Last 12 months, Top 50, All maps, Both sides

In [62]:
url = "https://www.hltv.org/stats/teams?startDate=2023-10-21&endDate=2024-10-21&rankingFilter=Top50"
response = scraper.get(url)

print(f"The status code is {response.status_code}")

The status code is 200


In [63]:
soup = bs(response.text)
tables = soup.find_all(class_="stats-table player-ratings-table")

df = pd.DataFrame(columns=["Team", "Maps", "K-D Diff", "K/D", "Rating"])

for row in tables[0].find_all('tr')[1:]:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]
    
    new_row = pd.DataFrame([cols], columns=df.columns)

    df = pd.concat([df, new_row], ignore_index=True, axis=0)

In [65]:
df.head()

,Team,Maps,K-D Diff,K/D,Rating
0,Vitality,140,+1025,1.11,1.12
1,Spirit,169,+1270,1.11,1.10
2,Natus Vincere,168,+805,1.07,1.07
3,MOUZ,164,+637,1.05,1.07
4,The MongolZ,153,+534,1.05,1.07


# Rondas de pistolas

All matches, Last 12 months, Top 50, All maps, Both sides

In [66]:
url = "https://www.hltv.org/stats/teams/pistols?startDate=2023-10-21&endDate=2024-10-21&rankingFilter=Top50"
response = scraper.get(url)

print(f"The status code is {response.status_code}")

The status code is 200


In [68]:
soup = bs(response.text)
tables = soup.find_all(class_="stats-table player-ratings-table ftu")

df = pd.DataFrame(columns=["Team", "Maps", "Won - Lost", "Pistol win %", "Round 2 conv", "Round 2 break"])

for row in tables[0].find_all('tr')[1:]:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]
    
    new_row = pd.DataFrame([cols], columns=df.columns)

    df = pd.concat([df, new_row], ignore_index=True, axis=0)

In [70]:
df.head()

,Team,Maps,Won - Lost,Pistol win %,Round 2 conv,Round 2 break
0,Spirit,169,185 - 153,54.7%,90.7%,11.1%
1,Vitality,140,152 - 128,54.3%,85.3%,31.2%
2,MOUZ,164,162 - 166,49.4%,90.1%,10.8%
3,The MongolZ,153,174 - 132,56.9%,83.7%,22.0%
4,Natus Vincere,168,189 - 147,56.2%,85.2%,12.2%


# FTU

(Firepower, Teamwork, Utility)

All matches, Last 12 months, Top 50, All maps, Both sides

In [82]:
url = "https://www.hltv.org/stats/teams/ftu?startDate=2023-10-21&endDate=2024-10-21&rankingFilter=Top50"
response = scraper.get(url)

print(f"The status code is {response.status_code}")

The status code is 200


In [102]:
soup = bs(response.text)
tables = soup.find_all(class_="stats-table player-ratings-table ftu gtSmartphone-only")

df = pd.DataFrame(columns=["Team", "Maps", "RW%", "OpK", "MultiK", "5v4%", "4v5%", "Traded%", "ADR", "FA"])

for row in tables[0].find_all('tr')[2:]:
    cols = row.find_all("td")
    cols = [col.text.strip() for col in cols]
    
    new_row = pd.DataFrame([cols], columns=df.columns)

    df = pd.concat([df, new_row], ignore_index=True, axis=0)

In [107]:
new_columns = pd.MultiIndex.from_tuples(
    [
        ("", "Team"), 
        ("", "Maps"),
        ("", "RW%"),
        ("Firepower", "OpK"),
        ("Firepower", "MultiK"),
        ('Teamwork', "5v4%"),
        ('Teamwork', "4v5%"),
        ('Teamwork', "Traded%"),
        ('Utility', "ADR"),
        ('Utility', "FA"),
    ]
)

df.columns = new_columns

df.head()

Firepower        Teamwork                Utility  \
            Team Maps    RW%       OpK MultiK     5v4%   4v5% Traded%     ADR   
0         Spirit  169  55.9%     53.4%   0.85    77.1%  31.7%   21.4%    24.6   
1       Vitality  140  55.8%     53.8%   0.84    75.4%  33.0%   22.4%    25.5   
2           MOUZ  164  53.3%     50.0%   0.84    75.8%  30.7%   20.4%    22.4   
3    The MongolZ  153  53.5%     49.7%   0.83    76.4%  30.9%   19.5%    24.5   
4  Natus Vincere  168  53.6%     53.3%   0.81    74.8%  29.6%   20.9%    21.5   

         
     FA  
0  0.19  
1  0.23  
2  0.24  
3  0.19  
4  0.24